In [ ]:
from sklearn.metrics import make_scorer
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.metrics import r2_score, mean_squared_error, d2_tweedie_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_is_fitted
from joblib import Parallel, delayed
import threading

import joblib
import numpy as np
import pandas as pd
import seaborn as sns

import bottleneck as bn

def NormRootMeanSqrtErr(y_true, y_pred, type = "minmax"):
    """
    Normalized Root Mean Square Error.
    
    interpretation: smaller is better.

    Args:
        y_true ([np.array]): test samples
        y_pred ([np.array]): predicted samples
        type (str): type of normalization. default is "minmax"
        - sd (standard deviation) : it divides the value by the standard deviation of the data.
        - mean (mean) : it divides the value by the mean of the data.
        - minmax (min-max) : it divides the value by the range of the data.
        - iqr (interquartile range) : it divides the value by the interquartile range of the data.

    Returns:
        [float]: normalized root mean square error
    """
    if type=="sd":
        return np.sqrt(mean_squared_error(y_true, y_pred))/np.std(y_true)
    elif type=="mean":
        return np.sqrt(mean_squared_error(y_true, y_pred))/np.mean(y_true)
    elif type=="minmax":
        return np.sqrt(mean_squared_error(y_true, y_pred))/(np.max(y_true) - np.min(y_true))
    elif type=="iqr":
        return np.sqrt(mean_squared_error(y_true, y_pred))/(np.quantile(y_true, 0.75) - np.quantile(y_true, 0.25))
    elif type!="":
        raise ValueError("type must be either 'sd', 'mean', 'minmax', or 'iqr'")

def concordance_correlation_coefficient(y_true, y_pred, wei):
  """Concordance correlation coefficient."""
  # Raw data
  dct = {
      'y_true': y_true,
      'y_pred': y_pred
  }
  df = pd.DataFrame(dct)
  # Remove NaNs
  df = df.dropna()
  # Pearson product-moment correlation coefficients
  y_true = df['y_true']
  y_pred = df['y_pred']
  #cor = np.corrcoef(y_true, y_pred)[0][1]
  cor = DescrStatsW(df.to_numpy(), weights=wei).corrcoef[0][1]
  # Means
  mean_true = np.average(y_true, weights=wei)
  mean_pred = np.average(y_pred, weights=wei)
  # Population variances
  #var_true = np.var(y_true)
  var_true = DescrStatsW(y_true, weights=wei).var
  #print(var_true, var_true1)
  #var_pred = np.var(y_pred)
  var_pred = DescrStatsW(y_pred, weights=wei).var
  # Population standard deviations
  sd_true = DescrStatsW(y_true, weights=wei).std
  sd_pred = DescrStatsW(y_pred, weights=wei).std
  # Calculate CCC
  numerator = 2 * cor * sd_true * sd_pred
  denominator = var_true + var_pred + (mean_true - mean_pred)**2

  return numerator / denominator

def _single_prediction(predict, X, out, i, lock):
    prediction = predict(X, check_input=False)
    with lock:
        out[i, :] = prediction

def cast_tree_rf(model):
    model.__class__ = TreesRandomForestRegressor
    return model

class TreesRandomForestRegressor(RandomForestRegressor):
    def predict(self, X):
        """
        Predict regression target for X.

        The predicted regression target of an input sample is computed according
        to a list of functions that receives the predicted regression targets of each 
        single tree in the forest.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            The input samples. Internally, its dtype will be converted to
            ``dtype=np.float32``. If a sparse matrix is provided, it will be
            converted into a sparse ``csr_matrix``.

        Returns
        -------
        s : an ndarray of shape (n_estimators, n_samples)
            The predicted values for each single tree.
        """
        check_is_fitted(self)
        # Check data
        X = self._validate_X_predict(X)

        # store the output of every estimator
        assert(self.n_outputs_ == 1)
        pred_t = np.empty((len(self.estimators_), X.shape[0]), dtype=np.float32)
        # Assign chunk of trees to jobs
        n_jobs = min(self.n_estimators, self.n_jobs)
        # Parallel loop prediction
        lock = threading.Lock()
        Parallel(n_jobs=n_jobs, verbose=self.verbose, require="sharedmem")(
            delayed(_single_prediction)(self.estimators_[i].predict, X, pred_t, i, lock)
            for i in range(len(self.estimators_))
        )
        return pred_t

ccc_scorer = make_scorer(concordance_correlation_coefficient, greater_is_better=True)

sns.set_theme(style="whitegrid", palette="pastel")
sns.set_context("notebook")

wd = '/mnt/tupi/WRI/livestock_global_modeling/livestock_census_ard'

sample_fn = f'{wd}/gpw_livestock.animals_gpw.fao.glw3_zonal.samples_20000101_20211231_go_epsg.4326_v1.pq'
#sample_fn = f'{wd}/gpw_livestock.animals_faostat_zonal.samples_20000101_20211231_go_epsg.4326_v1.pq'
samples = pd.read_parquet(sample_fn)

#animals = ['cattle', 'sheep', 'goat', 'horse', 'buffalo']
animals = ['cattle','sheep', 'goat', 'horse']
model_names = ['lgb', 'rf']
max_density = {
    'cattle': 1505,
    'sheep': 496,
    'goat': 292,
    'horse': 47,
    'buffalo': 129,
}

In [ ]:
polygon_samples = joblib.load('/mnt/tupi/WRI/internal-develop/gpw/livestock-modeling/faostat_livestock_all.lz4')
polygon_samples

In [ ]:
polygon_samples = joblib.load('/mnt/tupi/WRI/livestock_global_modeling/livestock_census_raw/gpw_livestock.animals_gpw.fao.glw3_polygon.samples_20000101_20231231_go_epsg.4326_v1.lz4')
polygon_samples

## Number of samples

In [ ]:
animals = ['cattle', 'sheep', 'goat', 'horse', 'buffalo']
stats = []
for a in animals:
    mask = np.logical_and(samples[f'ind_{a}'] == 1, samples[f'{a}_density'] <= max_density[a])
    omask = np.logical_and(samples[f'ind_{a}'] == 1, samples[f'{a}_density'] > max_density[a])
    zmask = np.logical_and(samples[f'ind_{a}'] == 1, samples[f'{a}_density'] == 0.001)
    samples[mask]['source'].unique()
    stats.append({
        'animal': a,
        'source': ', '.join(list(samples[mask]['source'].unique())),
        'q98': max_density[a],
        #'n_countries': len(samples[mask]['country'].unique()),
        'n_zeros': samples[zmask].shape[0],
        'n_outlier': samples[omask].shape[0],
        'n_samples': samples[mask].shape[0]
    })

pd.DataFrame(stats)#.to_csv('samples_stats.csv')

In [ ]:
samples[np.logical_and.reduce([
        samples['ind_cattle'] == 1,
        samples[f'{animal}_ml_type'] == 'testing',
        samples['cattle_density'] <= max_density['cattle'],
])][['cattle_density']].plot.hist(bins=128, title=f"{fn_model}.{model}", histtype='step', linewidth=1.5)

## Reading models

In [ ]:
model_names

In [ ]:
models = {}

#model_names = ['lgb', 'rf']
model_names = ['rf']

model_fn = 'zonal_models_ultimate'

#suffs = ['', '_boxcox']
suffs = ['']

for animal in animals:
  for mod in model_names:
    for s in suffs:

        fn_model = f'{animal}.{animal}_density{s}'

        wd = '/mnt/tupi/WRI/livestock_global_modeling/livestock_census_ard'

        #locals().update(**joblib.load(f'{wd}/{model_fn}/{fn_model}.{model}_cv.lz4'))
        locals().update(**joblib.load(f'{wd}/{model_fn}/{fn_model}.{mod}_prod.lz4'))
        locals().update(**joblib.load(f'{wd}/{model_fn}/{fn_model}_rfecv.lz4'))

        prod_mod = eval(f"prod_{mod}")
        if 'rf' in mod:
            prod_mod.__class__ = TreesRandomForestRegressor

        models[f'{animal}_{mod}{s}'] = {
            'rfe': covs_rfe,
            'prod': prod_mod,
            'pt': target_pt
        }

## Predict test set

In [ ]:
samples = samples.merge(polygon_samples[['gazID','geometry']], on='gazID')

In [ ]:
samples['area_km2'] = gpd.GeoSeries(samples['geometry']).to_crs('+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs').area / 1e6
samples['area_km2']

In [ ]:
import math

livestock_test = []

for animal in animals:
  for mod in model_names: #(model_names + ['eml']):
    for s in suffs:
    
        print(f"Runing model {animal}_{mod}{s}")

        samples_test = samples[np.logical_and.reduce([
            samples[f'ind_{animal}'] == 1 
            ,samples[f'{animal}_ml_type'] == 'testing'
        ])]
        
        
        ## Adding area
        #samples_test = samples_test.merge(polygon_samples[['gazID','geometry']], on='gazID')
        mask = (samples_test[f'{animal}_density'] <= max_density[animal])
        #mask = (samples_test[f'{animal}_density'] > 0)
        #samples_test['area_km2'] = 
        #print(samples_test.shape)
            
        covs_rfe = models[f'{animal}_{mod}{s}']['rfe']
        prod_mod = models[f'{animal}_{mod}{s}']['prod']
        pt = models[f'{animal}_{mod}{s}']['pt']

        pred = prod_mod.predict(samples_test[mask][covs_rfe])
        if 'rf' in mod:
            perc = np.nanpercentile(pred, [2.5,97.5], axis=0)
            std = (perc[1,:] - perc[0,:]) / 4
            pred = bn.nanmean(pred, axis=0)
        
        print(pred.shape, perc.shape)
        
        mod_row = pd.DataFrame({
          'livestock_area_km': samples_test[mask][f'livestock_area_km'],
          'predicted': pred,
          'expected': samples_test[mask][f'{animal}_density'],
          'predicted_p025': perc[0,:],
          'predicted_p975': perc[1,:],
          'predicted_lower': pred - std,
          'predicted_upper': pred + std,
          'weight': 1,
          'source': samples_test[mask]['source'],
          'gazName': samples_test[mask]['gazName'],
          'year': samples_test[mask]['year'],
          'model': f'{mod}{s}',
          'animal': animal
          #,'area_km2': samples_test[mask]['area_km2'] #gpd.GeoSeries(samples_test[mask]['geometry']).to_crs('+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs').area / 1e6
        })

        livestock_test.append(mod_row)
    
livestock_test = pd.concat(livestock_test)

In [ ]:
livestock_test['rf_inside_pr'] = np.logical_and.reduce([
    livestock_test['predicted_lower'] < livestock_test['expected'],
    livestock_test['predicted_upper'] > livestock_test['expected'],
])

In [ ]:
livestock_test[np.logical_not(livestock_test['rf_inside_pr'])]

In [ ]:
livestock_test

In [ ]:
livestock_test[['rf_inside_pr','animal']].rename(columns={
    'animal': 'Animals',
    'rf_inside_pr': 'RF'
}).groupby('Animals').mean().sort_values('RF').plot(kind='barh', xlabel="Prediction Interval Coverage Probability (PICP)", xlim=(0, 1)).legend(loc='best', bbox_to_anchor=(1, 1))

### Temporal validation

In [ ]:
for (year,animal), rows in livestock_test.groupby(['year', 'animal']):
    pred_label, expe_label = 'predicted', 'expected'
    rmse = mean_squared_error(rows[expe_label], rows[pred_label], squared=False)
    print(animal, year, rmse)

## Accuracy plots

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import colors
import math

import math
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib.ticker import LogFormatterExponent
from matplotlib import colors
import math

def plot_hexbin(livestock_rows_df, model, animals, pos, title, n_col_rows, figsize=(10,8)):

    fontsize = 14

    plt.rc('font', size=fontsize)          # controls default text sizes
    plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
    plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
    plt.rc('legend', fontsize=fontsize)    # legend fontsize
    plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title
    
    fig, axs = plt.subplots(ncols=n_col_rows[0], nrows=n_col_rows[1], figsize=figsize)
    #fig.suptitle(title)
    plt.tight_layout(pad=3, h_pad=4)

    for (animal,p) in zip(animals, pos):
        
        pred_label = f'Predicted density (heads per km2)'
        expe_label = f'Observed density (heads per km2)'
        
        col_remap = {}
        col_remap['predicted'] = pred_label
        col_remap['expected'] = expe_label
        
        livestock_rows_hexbin = livestock_rows_df[
            np.logical_and.reduce([
                livestock_rows_df['model'] == model,
                livestock_rows_df['animal'] == animal,
                livestock_rows_df['livestock_area_km'] > 0,
                livestock_rows_df['expected'] > 0,
                livestock_rows_df['predicted'] > 0,
                #livestock_rows_df['expected_density'] > 1
                #livestock_rows_df['source'].isin(['GPW']),
                #livestock_rows_df['source'].isin(['Malek et al., 2024']),
            ])
        ].rename(columns=col_remap)[[expe_label,pred_label]]

        max_log = math.ceil(np.max(np.log10(livestock_rows_hexbin.to_numpy())))
        print(max_log)

        d2 = d2_tweedie_score(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], power=1)
        nrmse = NormRootMeanSqrtErr(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], 'sd')
        rmse = mean_squared_error(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], squared=False)
        ccc = concordance_correlation_coefficient(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], 
                                            np.ones(livestock_rows_hexbin.shape[0]))
        r2 = r2_score(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label])
        stats=f'D2={d2:.3f}\nR2={r2:.3f}\nCCC={ccc:.3f}\nRMSE={rmse:.2f}\nNRMSE={nrmse:.2f}'

        livestock_rows_hexbin.plot.hexbin(x=pred_label, y=expe_label, mincnt=1, gridsize=48,  cmap="copper_r", 
                                               yscale='log', xscale='log', bins='log', extent=[-0.2,max_log,-0.2,max_log], ax=axs[p[0],p[1]])
        axs[p[0],p[1]].set_title(f'{animal.capitalize()} model ({title})', fontweight='bold')
        axs[p[0],p[1]].axline([0, 0], [1, 1], color='silver')
        axs[p[0],p[1]].text(0.05, 0.95, stats, transform=axs[p[0],p[1]].transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='square,pad=.6',
            facecolor=colors.to_rgba('white', alpha=0.8)))

model, model_label = 'rf', 'Random Forest'
title = f"{model_label}"
animals = ['cattle', 'horse', 'goat', 'sheep']
pos = [[0,0],[0,1],[1,0],[1,1]]
        
plot_hexbin(livestock_test, model, animals, pos, title, [2,2], figsize=(12,10))

In [ ]:
top10 = list(livestock_test.groupby('gazName').aggregate({'area_km2': 'first'}).sort_values('area_km2', ascending=False).head(10).reset_index()['gazName'])
top10

In [ ]:
livestock_test[np.logical_and.reduce([
    livestock_test['gazName'].isin(top10),
    livestock_test['animal'] == 'cattle',
    livestock_test['gazName'] == 'Algeria',
])][['year','gazName','expected','predicted']].set_index('year').plot(kind='line')

In [ ]:
livestock_test.query("animal == 'sheep' & gazName == 'Netherlands'")

## Variable importance

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme(style="whitegrid", palette="deep")
sns.set_context("notebook")

In [ ]:
#set(sum([ list(models[f'{a}_rf']['rfe']) for a in animals ],[]))
feat_labels = {
	'bare.rock_glc.gfz_p_1km_s_2015_2018_go_epsg4326_v20241017': 'Static bare rock (GLC/GFZ)',
	'bare.soil.sand_glc.gfz_p_1km_s_2015_2018_go_epsg4326_v20241017': 'Static bare sand (GLC/GFZ)',
	'bare.soil_mcd43a4.fc_m_500m_s_year0101_year1231_go_epsg.4326_v20240616': 'Annual bare soil fraction mean (MCD43A4)',
	'bare.soil_mcd43a4.fc_mx_500m_s_year0101_year1231_go_epsg.4326_v20240616': 'Annual bare soil fraction max. (MCD43A4)',
	'christianity.pct_world.religion_m_1km_s_20100101_20101231_go_epsg4326_v20241107': 'Christianity popuplation. pct. (World Religion Data)',
	'clm_accum.precipitation_chelsa.annual.log.csum_m_1km_s0..0cm_year_v2.1':  'Annual accumulated precipitation (CHELSA)',
	'clm_accum.precipitation_chelsa.annual_m_1km_s0..0cm_year_v2.1': 'Annual precipitation (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.01.01..year.01.31_v2.1': 'Monthly precipitation - Jan. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.02.01..year.02.28_v2.1': 'Monthly precipitation - Feb. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.05.01..year.05.31_v2.1': 'Monthly precipitation - May. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.06.01..year.06.30_v2.1': 'Monthly precipitation - Jun. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.07.01..year.07.31_v2.1': 'Monthly precipitation - Jul. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.08.01..year.08.31_v2.1': 'Monthly precipitation - Aug. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.09.01..year.09.30_v2.1': 'Monthly precipitation - Sep. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.10.01..year.10.31_v2.1': 'Monthly precipitation - Oct. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.11.01..year.11.30_v2.1': 'Monthly precipitation - Nov. (CHELSA)',
	'clm_accum.precipitation_chelsa.montlhy_m_1km_s0..0cm_year.12.01..year.12.31_v2.1': 'Monthly precipitation - Dec. (CHELSA)',
	'clm_lst_max.geom.temp_m_30m_s_m1': 'Long-term geometric max. temperature - Jan. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m10': 'Long-term geometric max. temperature - Oct. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m11': 'Long-term geometric max. temperature - Nov. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m12': 'Long-term geometric max. temperature - Dec. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m2': 'Long-term geometric max. temperature - Feb. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m3': 'Long-term geometric max. temperature - Mar. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m5': 'Long-term geometric max. temperature - May. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m6': 'Long-term geometric max. temperature - Jun. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m7': 'Long-term geometric max. temperature - Jul. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m8': 'Long-term geometric max. temperature - Aug. (Kilibarda et al., 2014)',
	'clm_lst_max.geom.temp_m_30m_s_m9': 'Long-term geometric max. temperature - Sep. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m1': 'Long-term geometric min. temperature - Jan. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m10': 'Long-term geometric min. temperature - Oct. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m11': 'Long-term geometric min. temperature - Nov. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m2': 'Long-term geometric min. temperature - Feb. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m3': 'Long-term geometric min. temperature - Mar. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m4': 'Long-term geometric min. temperature - Apr. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m5': 'Long-term geometric min. temperature - May. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m6': 'Long-term geometric min. temperature - Jun. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m7': 'Long-term geometric min. temperature - Jul. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m8': 'Long-term geometric min. temperature - Aug. (Kilibarda et al., 2014)',
	'clm_lst_min.geom.temp_m_30m_s_m9': 'Long-term geometric min. temperature - Sep. (Kilibarda et al., 2014)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.01.01..year.01.31_v1.2': 'Monthly day-time land surf. temp. trend - Jan. (MOD11A2)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.02.01..year.02.28_v1.2': 'Monthly day-time land surf. temp. trend - Feb. (MOD11A2)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.03.01..year.03.31_v1.2': 'Monthly day-time land surf. temp. trend - Mar. (MOD11A2)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.07.01..year.07.31_v1.2': 'Monthly day-time land surf. temp. trend - Jul. (MOD11A2)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.09.01..year.09.30_v1.2': 'Monthly day-time land surf. temp. trend - Sep. (MOD11A2)',
	'clm_lst_mod11a2.daytime.trend_p50_1km_s0..0cm_year.12.01..year.12.31_v1.2': 'Monthly day-time land surf. temp. trend - Dec. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.04.01..year.04.30_v1.2': 'Monthly day-time land surf. temp. - Apr. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.05.01..year.05.31_v1.2': 'Monthly day-time land surf. temp. - May (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.06.01..year.06.30_v1.2': 'Monthly day-time land surf. temp. - Jun. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.07.01..year.07.31_v1.2': 'Monthly day-time land surf. temp. - Jul. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.08.01..year.08.31_v1.2': 'Monthly day-time land surf. temp. - Aug. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.09.01..year.09.30_v1.2': 'Monthly day-time land surf. temp. - Sep. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.10.01..year.10.31_v1.2': 'Monthly day-time land surf. temp. - Oct. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.11.01..year.11.30_v1.2': 'Monthly day-time land surf. temp. - Nov. (MOD11A2)',
	'clm_lst_mod11a2.daytime_p50_1km_s0..0cm_year.12.01..year.12.31_v1.2': 'Monthly day-time land surf. temp. - Dec. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.01.01..year.01.31_v1.2': 'Monthly night-time land surf. temp. trend - Jan. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.02.01..year.02.28_v1.2': 'Monthly night-time land surf. temp. trend - Feb. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.04.01..year.04.30_v1.2': 'Monthly night-time land surf. temp. trend - Apr. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.07.01..year.07.31_v1.2': 'Monthly night-time land surf. temp. trend - Jul. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.10.01..year.10.31_v1.2': 'Monthly night-time land surf. temp. trend - Oct. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.11.01..year.11.30_v1.2': 'Monthly night-time land surf. temp. trend - Nov. (MOD11A2)',
	'clm_lst_mod11a2.nighttime.trend_p50_1km_s0..0cm_year.12.01..year.12.31_v1.2': 'Monthly night-time land surf. temp. trend - Dec. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.01.01..year.01.31_v1.2': 'Monthly night-time land surf. temp. - Jan. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.02.01..year.02.28_v1.2': 'Monthly night-time land surf. temp. - Feb. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.07.01..year.07.31_v1.2': 'Monthly night-time land surf. temp. - Jul. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.08.01..year.08.31_v1.2': 'Monthly night-time land surf. temp. - Aug. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.09.01..year.09.30_v1.2': 'Monthly night-time land surf. temp. - Sep. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.10.01..year.10.31_v1.2': 'Monthly night-time land surf. temp. - Oct. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.11.01..year.11.30_v1.2': 'Monthly night-time land surf. temp. - Nov. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year.12.01..year.12.31_v1.2': 'Monthly night-time land surf. temp. - Dec. (MOD11A2)',
	'clm_lst_mod11a2.nighttime_p50_1km_s0..0cm_year_v1.2': 'Annual night-time land surf. temp. (MOD11A2)',
	'easterness_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': "Static easterness (EDTM)",
	'filtered.dtm_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': "Static elevation (EDTM)",
	'flow.accum_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': "Static flow accumulation (EDTM)",
	'forest.cover_esa.cci_p_1km_s_year0101_year1231_go_epsg4326_v20241017': "Annual fraction of forest cover (ESA/CCI)",
	'gdp.per.capita.nat_sdata.kummu.2018_m_10km_s_year0101_year1231_go_epsg4326_v20241107': "Annual national GDP per capita (Kummu et al., 2018)",
	'gdp.per.capita_sdata.kummu.2018_m_10km_s_year0101_year1231_go_epsg4326_v20241107': "Annual sub-national GDP per capita (Kummu et al., 2018)",
	'geomorphon_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': "Static terrain form (EDTM)",
	'hdi_sdata.kummu.2018_m_10km_s_year0101_year1231_go_epsg4326_v20241107': "Annual Human Develop. Index (Kummu et al., 2018)",
	'hillshade_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': "Static hillshade (EDTM)",
	'islam.pct_world.religion_m_1km_s_20100101_20101231_go_epsg4326_v20241107': 'Islamic population pct. (World Religion Data)',
	'judaism.pct_world.religion_m_1km_s_20100101_20101231_go_epsg4326_v20241107': 'Judaism population pct. (World Religion Data)',
	'lcv_accessibility.to.cities_map.ox.var10_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (20k to 110 mi. pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var11_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (50k to 50 mi. pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var12_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (5k to 110 mi. pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var1_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (5—50 mi. pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var2_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (1—5 mi. pop - Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var3_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (500—1000k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var4_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (200—500k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var5_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (100—200k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var6_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (50—100k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var7_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (20—50k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var8_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (10—20k pop., Nelson et al., 2019)',
    'lcv_accessibility.to.cities_map.ox.var9_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. city (5—10k pop., Nelson et al., 2019)',
	'lcv_accessibility.to.ports_map.ox.var1_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. large port (Nelson et al. 2019)',
    'lcv_accessibility.to.ports_map.ox.var2_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. medium port (Nelson et al. 2019)',
    'lcv_accessibility.to.ports_map.ox.var3_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. small port (Nelson et al. 2019)',
    'lcv_accessibility.to.ports_map.ox.var4_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. very small port (Nelson et al. 2019)',
    'lcv_accessibility.to.ports_map.ox.var5_m_1km_s0..0cm_2015_v14052019': 'Travel time to near. any port (Nelson et al. 2019)',
	'lcv_water.occurance_jrc.surfacewater_p_250m_b0..200cm_1984..2018_v1.1': 'Long-term water occur. (JRC Global Surface Water)',
	'lcv_water.permanent_probav.lc100_p_250m_s0..0cm_2017_v1.0': 'Static permanent water (PROBAV water)',
	'lcv_wetlands.groundwater-driven_upmc.wtd_p_250m_b0..200cm_2010..2015_v1.0': 'Long-term groundwater-driven wetlands (Tootchi et al.,2019)',
	'lcv_wetlands.permanent_upmc.wtd_p_250m_b0..200cm_2010..2015_v1.0': 'Long-term permanent wetlands (Tootchi et al.,2019)',
	'lcv_wetlands.regularly-flooded_upmc.wtd_p_250m_b0..200cm_2010..2015_v1.0': 'Long-term regur. flooded wetlands (Tootchi et al.,2019)',
	'lf.bslope_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Landform backslope (EDTM)',
	'neg.openness_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Negative openness (EDTM)',
	'northerness_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Northerness (EDTM)',
	'nosink_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Nosink (EDTM)',
	'other.religions.pct_world.religion_m_1km_s_20100101_20101231_go_epsg4326_v20241107': 'Other religions population pct. (World Religion Data)',
	'pos.openness_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Positive openness (EDTM)',
	'rural.pop.dist_worldpop_m_1km_year0101_year1231_go_epsg4326_v20241129': 'Distance from rural population (WorldPop/GHS-SMOD)',
	'slope_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Slope (EDTM)',
	'spec.catch.area.factor_edtm_m_240m_s_20000101_20221231_go_epsg.4326_v20240528': 'Catchment area (EDTM)',
	'surface.water.fraction_dlr.gwp.annual_m_1km_s_year0101_year1231_go_epsg4326_v20241028': "Annual faction of surface water (Global WaterPack)",
	'surface.water.fraction_dlr.gwp.long.term_m_1km_s_20030101_20221231_go_epsg4326_v20241017': "Long-term faction of surface water (Global WaterPack)",
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.01.01..year.01.31_v1': 'Monthly blue reflectance - Jan. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.03.01..year.03.31_v1': 'Monthly blue reflectance - Mar. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.04.01..year.04.30_v1': 'Monthly blue reflectance - Apr. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Monthly blue reflectance - May (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.06.01..year.06.30_v1': 'Monthly blue reflectance - Jun. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.07.01..year.07.31_v1': 'Monthly blue reflectance - Jul. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year.12.01..year.12.31_v1': 'Monthly blue reflectance - Dec. (MOD13Q1)',
	'veg_blue_mod13q1.v061_p50_250m_s0..0cm_year_v1': 'Annual blue reflectance (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.02.01..year.02.28_v1': 'Monthly MIR reflectance - Feb. (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.03.01..year.03.31_v1': 'Monthly MIR reflectance - Mar. (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.04.01..year.04.30_v1': 'Monthly MIR reflectance - Apr. (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Monthly MIR reflectance - May (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.10.01..year.10.31_v1': 'Monthly MIR reflectance - Oct. (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year.11.01..year.11.30_v1': 'Monthly MIR reflectance - Nov. (MOD13Q1)',
	'veg_mir_mod13q1.v061_p50_250m_s0..0cm_year_v1': 'Annual MIR reflectance (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.01.01..year.01.31_v1': 'Monthly vegetation index trend (NDVI) - Jan. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.02.01..year.02.28_v1': 'Monthly vegetation index trend (NDVI) - Feb. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.03.01..year.03.31_v1': 'Monthly vegetation index trend (NDVI) - Mar. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Monthly vegetation index trend (NDVI) - May (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.07.01..year.07.31_v1': 'Monthly vegetation index trend (NDVI) - Jun. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.09.01..year.09.30_v1': 'Monthly vegetation index trend (NDVI) - Sep. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.10.01..year.10.31_v1': 'Monthly vegetation index trend (NDVI) - Oct. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061.trend_p50_250m_s0..0cm_year.12.01..year.12.31_v1': 'Monthly vegetation index trend (NDVI) - Dec. (MOD13Q1)',
	'veg_ndvi_mod13q1.v061_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Annual vegetation index (NDVI) - Jan. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.01.01..year.01.31_v1': 'Monthly NIR reflectance - Jan. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.02.01..year.02.28_v1': 'Monthly NIR reflectance - Feb. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.03.01..year.03.31_v1': 'Monthly NIR reflectance - Mar. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.04.01..year.04.30_v1': 'Monthly NIR reflectance - Apr. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Monthly NIR reflectance - May. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.06.01..year.06.30_v1': 'Monthly NIR reflectance - Jun. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.07.01..year.07.31_v1': 'Monthly NIR reflectance - Jul. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.08.01..year.08.31_v1': 'Monthly NIR reflectance - Aug. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.09.01..year.09.30_v1': 'Monthly NIR reflectance - Sep. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.10.01..year.10.31_v1': 'Monthly NIR reflectance - Oct. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.11.01..year.11.30_v1': 'Monthly NIR reflectance - Nov. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year.12.01..year.12.31_v1': 'Monthly NIR reflectance - Dec. (MOD13Q1)',
	'veg_nir_mod13q1.v061_p50_250m_s0..0cm_year_v1': 'Annual NIR reflectance (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.01.01..year.01.31_v1': 'Monthly red reflectance - Jan. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.02.01..year.02.28_v1': 'Monthly red reflectance - Feb. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.03.01..year.03.31_v1': 'Monthly red reflectance - Mar. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.04.01..year.04.30_v1': 'Monthly red reflectance - Apr. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.05.01..year.05.31_v1': 'Monthly red reflectance - May (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.08.01..year.08.31_v1': 'Monthly red reflectance - Aug. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.09.01..year.09.30_v1': 'Monthly red reflectance - Sep. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.11.01..year.11.30_v1': 'Monthly red reflectance - Nov. (MOD13Q1)',
	'veg_red_mod13q1.v061_p50_250m_s0..0cm_year.12.01..year.12.31_v1': 'Monthly red reflectance - Dec. (MOD13Q1)',
	'wilderness_li2022.human.footprint_p_1km_s_year0101_year1231_go_epsg.4326_v16022022': 'Annual wilderness, Human Footprint (Mu et al., 2022)',
	'wv_mcd19a2v061.seasconv.m.yearly_p25_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual water vapor - 25th perc. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.m.yearly_p75_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual water vapor - 75th perc. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.m.yearly_sd_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual water vapor - std. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.sd.yearly_p25_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual std. water vapor - 25th perc. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.sd.yearly_p50_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual std. water vapor - 50th perc. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.sd.yearly_p75_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual std. water vapor - 75th perc. (MCD19A2)',
	'wv_mcd19a2v061.seasconv.sd.yearly_sd_1km_s_year0101_year1231_go_epsg.4326_v20230619': 'Annual std. water vapor - std (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year0101_year0131_go_epsg.4326_v20230619': 'Monthly water vapor - Jan. (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year0701_year0731_go_epsg.4326_v20230619': 'Monthly water vapor - Jul. (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year0801_year0831_go_epsg.4326_v20230619': 'Monthly water vapor - Aug. (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year1001_year1031_go_epsg.4326_v20230619': 'Monthly water vapor - Oct. (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year1101_year1130_go_epsg.4326_v20230619': 'Monthly water vapor - Nov. (MCD19A2)',
	'wv_mcd19a2v061.seasconv_m_1km_s_year1201_year1231_go_epsg.4326_v20230619': 'Monthly water vapor - Dec. (MCD19A2)'
}

In [ ]:
def plot_var_imp(feature_cols, estimator, title, top_n = 20, figsize=(4,7), color = 'blue', model='rf'):
    
  if model == 'rf':
      var_imp = pd.DataFrame({'name':feature_cols, 'importance': estimator.feature_importances_})
  else:
      var_imp = pd.DataFrame({'name':feature_cols, 'importance': estimator.booster_.feature_importance(importance_type='gain')})
  
  var_imp['importance'] = var_imp['importance'] / var_imp['importance'].sum()
  var_imp.index = var_imp['name']
  result = var_imp.sort_values('importance', ascending=False)[0:top_n].sort_values('importance').plot(kind = 'barh', ylabel="", xlabel="Normalized importance", figsize=figsize, title = title, color = color)
  plt.title(title, fontweight='bold')
  return result

In [ ]:
max_log = math.ceil(np.max(np.log10(livestock_rows_hexbin.to_numpy())))
print(max_log)

livestock_rows_hexbin.plot.hexbin(x=pred_label, y=expe_label, mincnt=1, gridsize=48,  cmap="copper_r", 
                                       yscale='log', xscale='log', bins='log', extent=[-0.2,max_log,-0.2,max_log], ax=axs[p[0],p[1]])

### Partial Dependency Analysis (PDA)

In [ ]:
animal = 'cattle'

covs_rfe_rf = models[f'{animal}_rf']['rfe']
samples_test = samples[np.logical_and.reduce([
    samples[f'ind_{animal}'] == 1, 
    samples[f'{animal}_ml_type'] == 'testing'
])]
mask = (samples_test[f'{animal}_density'] <= max_density[animal])
X = samples_test[mask][covs_rfe_rf]

feature_cols = [ feat_labels[f].replace(' (',' \n(') for f in models[f'{animal}_rf']['rfe'] ]
estimator = models[f'{animal}_rf']['prod']
var_imp = pd.DataFrame({'name':feature_cols, 'importance': estimator.feature_importances_})

top_feat = list(var_imp.sort_values('importance', ascending=False).head(4)['name'])

### Partial Dep. plots

In [ ]:
animal = 'sheep'

covs_rfe_rf = models[f'{animal}_rf']['rfe']
samples_test = samples[np.logical_and.reduce([
    samples[f'ind_{animal}'] == 1, 
    samples[f'{animal}_ml_type'] == 'testing'
])]
mask = (samples_test[f'{animal}_density'] <= max_density[animal])
X = samples_test[mask][covs_rfe_rf]

feature_cols = [ feat_labels[f].replace(' (',' \n(') for f in models[f'{animal}_rf']['rfe'] ]
estimator = models[f'{animal}_rf']['prod']
var_imp = pd.DataFrame({'name':feature_cols, 'importance': estimator.feature_importances_})

top_feat = list(var_imp.sort_values('importance', ascending=False).head(4)['name'])

In [ ]:
# 'Annual fraction of forest cover (ESA/CCI)'
# Monthly MIR reflectance - Oct. (MOD13Q1)

fontsize = 14

plt.rc('font', size=fontsize)          # controls default text sizes
plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('legend', fontsize=fontsize)    # legend fontsize
plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title

title = f'{animal.capitalize()} model (Random Forest)'

fig, ax = plt.subplots(figsize=(15, 4))
plt.tight_layout()
ax.set_title(title, fontweight='bold')

pdd = PartialDependenceDisplay.from_estimator(models[f'{animal}_rf']['prod'], 
                                              X, 
                                              top_feat, 
                                              n_cols=4,
                                              subsample=None, feature_names=feature_cols, ax=ax)
fig.savefig(f"{animal}_rf_pda_top4.pdf", bbox_inches='tight')

In [ ]:
fontsize = 14

plt.rc('font', size=fontsize)          # controls default text sizes
plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
plt.rc('legend', fontsize=fontsize)    # legend fontsize
plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(12,10))
#fig.suptitle(title)
#plt.tight_layout(pad=3, h_pad=4)
pos = [[0,0],[0,1],[1,0],[1,1]]

for p, a in zip(pos,animals):
    feature_cols = [ feat_labels[f] for f in models[f'{a}_rf']['rfe'] ]
    estimator = models[f'{a}_rf']['prod']
    title = f'{a.capitalize()} model (Random Forest)'
    color = '#bf794d'
    top_n = 20
    
    var_imp = pd.DataFrame({'name':feature_cols, 'importance': estimator.feature_importances_})
    var_imp['importance'] = var_imp['importance'] / var_imp['importance'].sum()
    var_imp.index = var_imp['name']
    result = var_imp.sort_values('importance', ascending=False)[0:top_n].sort_values('importance').plot(kind = 'barh', ylabel="", 
                xlabel="Normalized importance", title = title, color = color,
                ax=axs[p[0],p[1]])
    

## Model benchmark

In [ ]:
find_files(wd, 'test*v20250216.pq')

In [ ]:
from skmap.misc import find_files

df_bm = pd.concat([ pd.read_parquet(f) for f in find_files(wd, 'test*.pq') ])
df_bm = df_bm[df_bm['model'].isin(['lgb','lgb_boxcox','rf','rf_boxcox'])]
df_bm['model'] = df_bm['model'] + df_bm['strategy'].map({
    'zonal_models_zeros_nowei_prod_v20250216': '',
    'zonal_models_zeros_wei_prod_v20250216': '_weight'
})
df_bm['label'] = df_bm['model'].map({
    'lgb': 'GBT',
    'lgb_boxcox': 'GBT / BoxCox trans.',
    'rf': 'RF',
    'rf_boxcox': 'RF / BoxCox Trans.',
    'lgb_weight': 'GBT / Sample weights',
    'lgb_boxcox_weight': 'GBT / BoxCox trans. / Sample weights',
    'rf_weight': 'RF / Sample weights',
    'rf_boxcox_weight': 'RF / BoxCox trans. / Sample weights'
})
df_bm

In [ ]:
mask = np.logical_and.reduce([
    df_bm['livestock_area_km'] > 0,
    df_bm['expected'] > 0,
    df_bm['predicted'] > 0
])

In [ ]:
results = []
for (label, animal), frows in df_bm[mask].groupby(['label','animal']):
    expe_label, pred_label = 'expected_density', 'predicted_density'
    results.append({
        'label': label,
        'animal': animal,
        'd2': d2_tweedie_score(frows[expe_label], frows[pred_label], power=1),
        'rmse': mean_squared_error(frows[expe_label], frows[pred_label], squared=False),
        'ccc': concordance_correlation_coefficient(frows[expe_label], frows[pred_label], np.ones(frows.shape[0])),
        'r2': r2_score(frows[expe_label], frows[pred_label])
    })

results = pd.DataFrame(results)

In [ ]:
results['label'] = results['label'].replace({
    'GBT / BoxCox trans. / Sample weights': 'GBT / BoxCox trans. / weighed samples',
    'GBT / Sample weights': 'GBT / weighed samples',
    'RF / BoxCox trans. / Sample weights': 'RF / BoxCox trans. / weighed samples',
    'RF / Sample weights': 'RF / weighed samples',
    'RF': 'RF',
    
})

In [ ]:
results[['animal','label','d2','r2','ccc','rmse']].round(3).sort_values(['animal','label'], ascending=True).to_csv('benchmark.csv')

In [ ]:
results.groupby('animal').agg('label')

In [ ]:
animal = 'horse'
livestock_test[np.logical_and.reduce([
        livestock_test['animal'] == animal,
        livestock_test['model'] == 'rf',
        livestock_test['expected_density'] > 1,
        livestock_test['expected_density'] <= max_density[animal]
])][['expected_density','predicted_density']]\
.plot.hist(bins=128,  histtype='step', linewidth=1.5)

In [ ]:
for source, livestock_rows_hexbin in livestock_test[np.logical_and.reduce([
        livestock_test['animal'] == 'cattle',
        livestock_test['source'] == 'GPW',
        livestock_test['model'] == 'rf',
    ])].groupby('country'):
    print(source, livestock_rows_hexbin.shape)
    
    pred_label = 'predicted'
    expe_label = 'expected' 
    
    livestock_rows_hexbin_log = pd.DataFrame(livestock_rows_hexbin)
    livestock_rows_hexbin_log[expe_label] = np.log1p(livestock_rows_hexbin[expe_label])
    livestock_rows_hexbin_log[pred_label] = np.log1p(livestock_rows_hexbin[pred_label])
    
    d2 = d2_tweedie_score(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], power=1)
    nrmse = NormRootMeanSqrtErr(livestock_rows_hexbin[expe_label], livestock_rows_hexbin[pred_label], 'sd')
    ccc = concordance_correlation_coefficient(livestock_rows_hexbin_log[expe_label], livestock_rows_hexbin_log[pred_label], 
                                        np.ones(livestock_rows_hexbin.shape[0]))
    #r2 = r2_score(livestock_rows_hexbin_log[expe_label], livestock_rows_hexbin_log[pred_label])
    stats=f'D2={d2:.3f} NRMSE={nrmse:.3f} CCC(log1p)={ccc:.3f}'
    print(stats)

In [ ]:
import math
for animal in animals:
  for model in models:

    #animal = 'cattle'
    #model = 'rf' #'lgb'
    fn_model = f'{animal}.{animal}_density'

    wd = '/mnt/tupi/WRI/livestock_global_modeling/livestock_census_ard'

    #locals().update(**joblib.load(f'{wd}/zonal_models_prod_v20250203/{fn_model}.{model}_cv.lz4'))
    locals().update(**joblib.load(f'{wd}/zonal_models_prod_v20250203/{fn_model}.{model}_prod.lz4'))
    locals().update(**joblib.load(f'{wd}/zonal_models_prod_v20250203/{fn_model}_rfecv.lz4'))

    prod_mod = eval(f"prod_{model}")

    samples_test = samples[np.logical_and(samples[f'ind_{animal}'] == 1, samples[f'{animal}_ml_type'] == 'testing')]
    
    if 'boxcox' in fn_model:
      df_cv['predicted'] = target_pt.inverse_transform(df_cv['predicted'].to_numpy().reshape(-1,1))
      df_cv['expected'] = target_pt.inverse_transform(df_cv['expected'].to_numpy().reshape(-1,1))

    mask = (samples_test[f'{animal}_density'] <= max_density[animal])
    df_test = pd.DataFrame({
      'predicted': prod_mod.predict(samples_test[mask][covs_rfe])  * samples_test[mask][f'livestock_area_km'],
      'expected':  samples_test[mask][f'{animal}_density'] * samples_test[mask][f'livestock_area_km'],
      'weight': samples_test[mask]['weight']
    })

    if 'boxcox' in fn_model:
      min_transformed_val = -2.65610344
      df_test.loc[df_test['predicted'] < min_transformed_val, 'predicted'] = min_transformed_val
      df_test['predicted'] = target_pt.inverse_transform(df_test['predicted'].to_numpy().reshape(-1,1))
    
    max_hist = np.percentile(df_test[['expected','predicted']].to_numpy(),[99])[0]
    mask_hist = np.logical_and(df_test['expected'] <= max_hist, df_test['predicted'] <= max_hist)
    
    plot = df_test[mask_hist][['predicted','expected']].plot.hist(bins=128, title=f"{fn_model}.{model}", histtype='step', linewidth=1.5, log=True)
    fig = plot.get_figure()
    #fig.savefig(f"{wd}/models/{fn_model}.{model}_test.log.hist.png")
    
    #plot = df_test_agg[['predicted','expected']].plot.hist(bins=128, title=f"{fn_model}.{model}", histtype='step', linewidth=1.5, density=1, stacked=False)
    #fig = plot.get_figure()
    #fig.savefig(f"{wd}/models/{fn_model}.{model}_test.hist.png")
    #print(f"{wd}/models/{fn_model}.{model}_test.hist.png")
    df_test['weight'] = 1
    #r2 = r2_score(df_test['expected'], df_test['predicted'], sample_weight=df_test['weight'])
    d2 = d2_tweedie_score(df_test['expected'], df_test['predicted'], power=1)
    rmse = mean_squared_error(df_test['expected'], df_test['predicted'], sample_weight=df_test['weight'], squared=False)
    
    #ccc = concordance_correlation_coefficient(df_test['expected'], df_test['predicted'], df_test['weight'])
    
    max_log = math.ceil(np.log10(np.max(df_test[['expected','predicted']].to_numpy())))
    
    #plot = df_test.plot.hexbin(x='expected', y='predicted', mincnt=1, gridsize=48,  cmap="Spectral_r", extent=[-0.2,max_log,-0.2,max_log],
    #                        title=f'{fn_model}.{model} \nd2={d2:.3f} rmse={rmse:.3f}', 
    #                             yscale='log', xscale='log', bins='log')
    #plot.axline([0, 0], [1, 1], color='silver')
    #fig = plot.get_figure()
    #fig.savefig(f"{wd}/models/{fn_model}.{model}_test.hebin.png")

    #ax.set_aspect('equal', 'datalim', share=True)
    #ax.set_aspect(1)

    plot = plot_var_imp(covs_rfe, prod_mod, f'{fn_model}.{model}', 40, color='#6fa8dd', model=model)
    fig = plot.get_figure()
    #fig.savefig(f"{wd}/models/{fn_model}.{model}_var.imp.png", bbox_inches="tight")

    

## Point time-series

In [ ]:
y,x = -8.96941598, -64.01426078 #-9.04264281, -63.45872163

In [ ]:
df = pd.DataFrame({
    'x': x,
    'y': y,
    'year': range(y1, y2+1) 
})
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326"
).to_crs('+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs')
#gdf['dt'] = pd.to_datetime(df['year'],format='%Y')

In [ ]:
import rasterio
def sample(row):
    y = row['year']
    
    ds = rasterio.open(f'https://s3.opengeohub.org/gpw/livestock/gpw_cattle.density_rf_m_1km_s_{y}0101_{y}1231_go_epsg.4326_v1.tif')
    row['density'] = next(ds.sample([(row["geometry"].x,row["geometry"].y)]))[0] * 0.1
    
    ds1 = rasterio.open(f'https://s3.opengeohub.org/gpw/livestock/gpw_cattle.density_rf_p.025_1km_s_{y}0101_{y}1231_go_epsg.4326_v1.tif')
    row['density_p025'] = next(ds1.sample([(row["geometry"].x,row["geometry"].y)]))[0] * 0.1
    
    ds3 = rasterio.open(f'https://s3.opengeohub.org/gpw/livestock/gpw_cattle.density_rf_p.975_1km_s_{y}0101_{y}1231_go_epsg.4326_v1.tif')
    row['density_p975'] = next(ds3.sample([(row["geometry"].x,row["geometry"].y)]))[0] * 0.1
    
    std = (row['density_p975'] - row['density_p025']) / 4
    row['lower'] = row['density'] - std
    row['upper'] = row['density'] + std
    
    return row

result = gdf.apply(sample, axis=1)

In [ ]:
((result['density_p975'] - result['density_p025']) / 4).mean()

In [ ]:
import seaborn 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

sns.set_style("ticks")
FONT_SIZE = 25
plt.rc('font', size=FONT_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=FONT_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=FONT_SIZE)    # legend fontsize
plt.rc('figure', titlesize=FONT_SIZE) 

myfig, myax = plt.subplots(figsize=(24, 6))

# Plot temperature
myax.plot(result['year'], result['density'], color='tab:red', linestyle='-', label='Cattle density')
myax.fill_between(result['year'], result['lower'], result['upper'], color='tab:red', alpha=0.2, label='Prediction error (1 SD)')
myax.fill_between(result['year'], result['density_p025'], result['density_p975'], color='tab:blue', alpha=0.2, label='Prediction interval (95% prob.)')
#myax.plot(pd.to_datetime(gpf['start_date'], format='%Y-%m-%d'), gpf['value']/10000, color='tab:blue', linestyle='-', label='Gapfilled')
#myax.plot(pd.to_datetime(stl['start_date'], format='%Y-%m-%d'), stl['value']/10000, color='tab:green', linestyle='-', label='Trend')

myax.set_ylim([0,900])
myax.set_ylabel('Heads per km-square')
myax.set_title(f'Coordinate: {x:.4f}, {y:.4f}; average cattle density: {result["density"].mean():.1f}')
myax.grid(True)

# format x axis labels
#myax.xaxis.set_major_locator(DayLocator())
#myax.xaxis.set_major_formatter(mdates.DateFormatter('%y%m%d'))
#fmt_half_year = mdates.MonthLocator(interval=29)

myax.legend(loc='upper right');

In [ ]:
import geopandas as gpd
import pandas as pd

wd = '/mnt/tupi/WRI/livestock_global_modeling'

polygons = gpd.read_file(f'{wd}/livestock_census_ard/gpw_livestock.animals_gpw.fao.glw3_polygon.samples_20000101_20231231_go_epsg.4326_v1.gpkg')
polygons

In [ ]:
import pandas as pd
import numpy as np

grassland_size = pd.read_csv(f'{wd}/grassland_size.csv')
grassland_pols = grassland_size.join(polygons['geometry'])

animals = ['cattle', 'sheep', 'goat', 'horse', 'buffalo']
livestock_all_rows = []

for animal in animals:
    for y in range(2000, 2021):
        col = f'{animal}_{y}'
        gcol = f'grassland_w_{y}'
        mask = ~np.isnan(grassland_pols[col])
        cols = ['gazID', 'gazName', 'source', 'geometry', 'total_pixels', col, gcol]
        rcols = {}
        rcols[col] = 'total'
        rcols[gcol] = 'grassland_area'
        animal_rows = grassland_pols[mask][cols].rename(columns=rcols)
        animal_rows['animal'] = animal
        animal_rows['year'] = y
        livestock_all_rows.append(animal_rows)
        print(col, animal_rows.shape)
        
livestock_all_rows = pd.concat(livestock_all_rows).reset_index(drop=True)
#livestock_rows['index'] = livestock_rows.index
#livestock_rows = livestock_rows.to_crs('ESRI:54052')
livestock_all_rows

In [ ]:
livestock_all_rows['source'].value_counts()

In [ ]:
def q01(x):
    return x.quantile(0.01)
def q02(x):
    return x.quantile(0.02)
def q03(x):
    return x.quantile(0.03)
def q04(x):
    return x.quantile(0.04)
def q05(x):
    return x.quantile(0.05)

def q95(x):
    return x.quantile(0.95)
def q96(x):
    return x.quantile(0.96)
def q97(x):
    return x.quantile(0.97)
def q98(x):
    return x.quantile(0.98)
def q99(x):
    return x.quantile(0.99)

livestock_all_rows[np.logical_and.reduce([
    livestock_all_rows['source'].isin(['GPW','fao_glw3']),
    livestock_all_rows['grassland_pct'] >= 0.02,
    #livestock_all_rows['grassland_area'] > 1000,
    livestock_all_rows['total'] >= 100,
])][['density','animal']].groupby('animal').agg({'density': ['min', q01, q02, q03, q04, q05, 'mean', 'median', q95, q96, q97, q98, q99, 'max']}).T.round(3)
#.groupby(['animal']).transform("count")
#.plot(kind='hist',bins=128, log=True, histtype='step')
#['density'].plot(kind='hist',bins=128, log=True)

In [ ]:
filters = [
    ('cattle', 1965),
    ('buffalo', 298.0),
    ('goat', 177),
    ('horse', 27),
    ('sheep', 178),
]

min_grassland_pct = 0.02
min_n_animals = 1000

In [ ]:
tgt_cols = pd.Index(['gazName','source','min_density','max_density','n_years','min_grassland_area','max_grassland_area','min_n_animals','max_n_animals','min__grassland_pct','max_grassland_pct'])

for animal, max_density in filters:
    cols = list(livestock_all_rows.columns[1:3])

    mask = np.logical_or.reduce([
        np.logical_and.reduce([
            livestock_all_rows['source'] == 'FAOSTAT',
            livestock_all_rows['animal'] == animal
        ]),
        np.logical_and.reduce([
            livestock_all_rows['source'].isin(['GPW','fao_glw3']),
            livestock_all_rows['grassland_pct'] >= min_grassland_pct,
            livestock_all_rows['density'] <= max_density,
            livestock_all_rows['animal'] == animal,
            livestock_all_rows['total'] >= min_n_animals
        ])
    ])
    #print(min_grassland_pct, max_density, animal, min_n_animals, livestock_all_rows[mask].shape)
    grassland_systems = livestock_all_rows[mask].groupby(cols).aggregate({
        'density': ['min', 'max'], 
        'year': 'count', 
        'grassland_area': ['min', 'max'], 
        'total': ['min', 'max'],
        'grassland_pct': ['min', 'max']
    }).reset_index()
    grassland_systems.columns = tgt_cols
    print(f"{animal} grassland_systems: {grassland_systems.shape}")
    
    landless_mask = np.logical_and.reduce([
        livestock_all_rows['animal'] == animal,
        livestock_all_rows['total'] >= min_n_animals,
        livestock_all_rows['density'] > max_density,
        ~livestock_all_rows['gazName'].isin(grassland_systems['gazName'])
    ])
    
    landless_systems = livestock_all_rows[landless_mask].groupby(cols).aggregate({
        'density': ['min', 'max'], 
        'year': 'count', 
        'grassland_area': ['min', 'max'], 
        'total': ['min', 'max'],
        'grassland_pct': ['min', 'max']
    }).reset_index()
    landless_systems.columns = tgt_cols
    print(f"{animal} landless_systems: {landless_systems.shape}")
    
    removed_mask = np.logical_and.reduce([
        livestock_all_rows['animal'] == animal,
        ~livestock_all_rows['gazName'].isin(grassland_systems['gazName']),
        ~livestock_all_rows['gazName'].isin(landless_systems['gazName'])
    ])
    
    removed = livestock_all_rows[removed_mask].groupby(cols).aggregate({
        'density': ['min', 'max'], 
        'year': 'count', 
        'grassland_area': ['min', 'max'], 
        'total': ['min', 'max'],
        'grassland_pct': ['min', 'max']
    }).reset_index()
    removed.columns = tgt_cols
    print(f"{animal} removed: {removed.shape}")
    
    grassland_systems = polygons[['gazID', 'gazName','source','geometry']].merge(grassland_systems, on=['gazName','source'])
    grassland_systems.loc[:,'category'] = 'Grazing systems'
    
    landless_systems = polygons[['gazID', 'gazName','source','geometry']].merge(landless_systems, on=['gazName','source'])
    landless_systems.loc[:,'category'] = 'Non-grazing systems'
    
    removed = polygons[['gazID', 'gazName','source','geometry']].merge(removed, on=['gazName','source'])
    removed.loc[:,'category'] = 'Ignored'
    
    pd.concat([
        grassland_systems,
        landless_systems,
        removed
    ]).to_file(f'gpw_livestock.systems.category_{animal}.gpkg')
    
    #polygons[['gazID', 'gazName','source','geometry']].merge(grassland_systems, on=['gazName','source']).to_file(f'livestock.grassland.{animal}.gpkg')
    #polygons[['gazID', 'gazName','source','geometry']].merge(landless_systems, on=['gazName','source']).to_file(f'livestock.landless.{animal}.gpkg')
    #polygons[['gazID', 'gazName','source','geometry']].merge(removed, on=['gazName','source']).to_file(f'livestock.removed.{animal}.gpkg')

In [ ]:
animal = 'cattle'
#livestock_systems = gpd.read_file(f'gpw_livestock.systems.category_{animal}.gpkg')


In [ ]:
livestock_systems_map = {}
for animal,_ in filters:
    livestock_systems_map[animal] = gpd.read_file(f'gpw_livestock.systems.category_{animal}.gpkg')

In [ ]:
animal = 'goat'
#for animal,_ in filters:
print(f"#####: {animal}")
livestock_systems = livestock_systems_map[animal]
livestock_systems['category'].value_counts().plot(kind='pie', colors=['#ffe201','#ff0105','#01f3ff'], title='Number of polygons')
livestock_systems['polygon_area_km'] = livestock_systems.to_crs('ESRI:54052').geometry.area / 1000000
livestock_systems.groupby('category').aggregate({'polygon_area_km': 'sum'}).plot(kind='pie', y='polygon_area_km', colors=['#ffe201','#ff0105','#01f3ff'], title='Area of polygons',legend=False)

In [ ]:
polygons['gazID'].isnull().value_counts()a

In [ ]:
len(polygons['gazName'].unique())

In [ ]:
polygons[['gazID', 'gazName','source','geometry']].merge(livestock_all_rows[landless_mask], on=['gazName','source']).to_file(f'livestock.landless.{animal}.gpkg')

In [ ]:
polygons[['gazID', 'gazName','source','geometry']].merge(livestock_all_rows[landless_mask], on=['gazName','source'])

In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid", palette="pastel")
sns.set_context("notebook")

livestock_all_rows[livestock_all_rows['grassland_pct'] < 0.1]['grassland_pct'].plot(kind='hist',bins=128, log=True)

In [ ]:
#livestock_all_rows.loc[:,['grassland_pct']] = livestock_all_rows['grassland_area'] / livestock_all_rows['total_pixels']
#livestock_all_rows.loc[:,['density']] = livestock_all_rows['total'] / livestock_all_rows['grassland_area']
livestock_all_rows
#livestock_all_rows[livestock_all_rows['grassland_area'] > livestock_all_rows['total_pixels']]

In [ ]:
import pandas as pd
grassland_size = pd.read_csv(f'{wd}/grassland_size.csv')

animals = ['cattle', 'sheep', 'goat', 'horse', 'buffalo']

for animal in animals:
    livestock_test = pd.read_csv(f'{wd}/../{animal}_validation_set_v2.csv')
    livestock_test.loc[:,'index'] = livestock_test.index
    
    animal_cols = livestock_test.columns.drop(
        sum([ list(livestock_test.columns[livestock_test.columns.str.contains(a)]) for a in animals if a != animal],[])
    )
    
    livestock_test = polygons[['gazName', 'source', 'geometry']].merge(livestock_test[animal_cols], on=['gazName', 'source'])
    print(f"Saving test polygons for {animal} ({livestock_test.shape})")
    
    livestock_test.to_file(f'{wd}/{animal}_validation_set_v2.gpkg')

In [ ]:
#import pandas as pd
#import geopandas as gpd
#livestock_test = gpd.read_file(f'{wd}/livestock_validation_set.gpkg')

In [ ]:
import numpy as np

animals = ['cattle', 'sheep', 'goat', 'horse', 'buffalo']
livestock_rows = []

for animal in animals:
    livestock_test = gpd.read_file(f'{wd}/{animal}_validation_set_v2.gpkg')
    
    for y in range(2000, 2021):
        col = f'{animal}_{y}'
        gcol = f'grassland_w_{y}'
        mask = ~np.isnan(livestock_test[col])
        cols = ['gazID', 'gazName', 'source', 'geometry', 'total_pixels', col, gcol]
        rcols = {}
        rcols[col] = 'total'
        rcols[gcol] = 'grassland_area'
        animal_rows = livestock_test[mask][cols].rename(columns=rcols)
        animal_rows['animal'] = animal
        animal_rows['year'] = y
        livestock_rows.append(animal_rows)
        print(col, animal_rows.shape)
        
livestock_rows = pd.concat(livestock_rows).reset_index(drop=True)
livestock_rows['index'] = livestock_rows.index
livestock_rows = livestock_rows.to_crs('ESRI:54052')
livestock_rows

In [ ]:
#livestock_polygon = livestock_rows[np.logical_and(livestock_rows['animal'] == 'cattle', livestock_rows['year'] == 2020)]
#livestock_polygon

In [ ]:
from rasterstats import zonal_stats
import geopandas as gpd

def run_zonal(rows, animal, year):
    models = [ 'rf_m', 'lgb_m', 'eml_p.025', 'eml_m', 'eml_p.975' ]
    
    out = rows[['index','geometry']]
    try:
        for m in models:
            out = gpd.GeoDataFrame.from_features(
                    zonal_stats(
                    out, 
                    f"/vsicurl/http://192.168.49.30:8333/gpw/arco/gpw_{animal}.total_{m}_1km_s_{year}0101_{year}1231_go_esri.54052_v20240809.tif",
                    prefix=f"{m}",
                    stats="sum", 
                    geojson_out=True)
            )
    except:
        pass
    
    return out

In [ ]:
args = []

for (animal, year), rows in livestock_rows[livestock_rows['animal'].isin(animals)].groupby(['animal','year']):
    args.append((rows, animal, year,))

len(args)

In [ ]:
from eumap import parallel
from skmap.misc import ttprint

ttprint("Start zonal")
df = pd.concat([ df for df in parallel.job(run_zonal, args, n_jobs=40, joblib_args={'backend': 'multiprocessing'}) ])
ttprint("End zonal")

In [ ]:
livestock_rows_df = pd.merge(livestock_rows, df, on='index')

In [ ]:
raster_cols = livestock_rows_df.columns[livestock_rows_df.columns.str.contains('sum')]
livestock_rows_df.loc[:,raster_cols] = livestock_rows_df[raster_cols]  / 10
livestock_rows_df

In [ ]:
livestock_rows_df['inside_pred_interval'] = np.logical_and.reduce([
    livestock_rows_df['total'] <= livestock_rows_df['eml_p.975sum'],
    livestock_rows_df['total'] >= livestock_rows_df['eml_p.025sum'] 
])

In [ ]:
livestock_rows_df['density'] = livestock_rows_df['total'] / livestock_rows_df['grassland_area']

In [ ]:
livestock_rows_df.drop(columns=['geometry_x', 'geometry_y']).to_parquet(f'{wd}/validation_set_v2_zonal.pq')

In [ ]:
#livestock_rows_df = pd.read_parquet(f'{wd}/livestock_validation_set_zonal.pq')
#livestock_rows_df = livestock_rows_df.drop(columns=['geometry_x', 'geometry_y'])
#livestock_rows_df

In [ ]:
#livestock_rows_df['density'] = livestock_rows_df['total'] / livestock_rows_df['grassland_area']

In [ ]:
#predicted_col = 'eml_msum'
#expected_col = 'total'
#livestock_rows_df[livestock_rows_df['animal'] == 'cattle'][['total','rf_msum']]

In [ ]:
#livestock_rows_df['inside_pred_interval'].value_counts()

In [ ]:
#[livestock_rows_df['gazID'] == '.002.003.003.022.019']

nrs = []
for gazName, rows in livestock_rows_df.groupby('gazName'):
    nr = {}
    for (year, animal), r in rows.groupby(['year', 'animal']):
        for m in ['eml_m', 'eml_p.025', 'eml_p.975']:
            nr[f'{animal}_{year}_{m}'] = r[f'{m}sum'].iloc[0]
        nr[f'{animal}_{year}_picp'] = (r['inside_pred_interval'].iloc[0]).astype('int')
        nr[f'{animal}_{year}_density'] = r['density'].iloc[0]
        nr[f'gazName'] = r['gazName'].iloc[0]
    nrs.append(nr)
    
nrs = pd.DataFrame(nrs)
nrs

In [ ]:
nrs[['gazName'] + sorted(list(nrs.columns.drop(['gazName'])))]

In [ ]:
#import geopandas as gpd
#validation_set = gpd.read_file(f'{wd}/livestock_validation_set.gpkg')
#validation_set

In [ ]:
livestock_test_nrs = livestock_test.merge(nrs, on='gazName')
for a in animals:
    livestock_test_nrs.loc[:,f'{a}_test'] = (np.nansum(livestock_test_nrs[livestock_test_nrs.columns[livestock_test_nrs.columns.str.contains(f'{a}_.*_density')]].to_numpy(), axis=-1) > 0).astype('int')

for a in animals:
    livestock_test_nrs.loc[:,f'{a}_picp'] = np.nansum(livestock_test_nrs[livestock_test_nrs.columns[livestock_test_nrs.columns.str.contains(f'{a}_.*_picp')]].to_numpy(), axis=-1)

In [ ]:
cols = list(livestock_test_nrs.columns[:3]) + sorted(list(livestock_test_nrs.columns[3:]))

In [ ]:
list(livestock_test_nrs[cols].columns)

In [ ]:
livestock_test_nrs[cols].to_file(f'{wd}/validation_set_v2_zonal.gpkg')

In [ ]:
mask = np.logical_and.reduce([
    livestock_rows_df['animal'] == 'goat',
    livestock_rows_df['source'] == 'GPW',
    livestock_rows_df['total'] < 500
])


In [ ]:
blunders = (livestock_rows_df[mask]['total'] - livestock_rows_df[mask]['eml_msum']).sort_values().head(10).index
livestock_rows_df.loc[blunders,:]

In [ ]:
livestock_rows.loc[blunders,:].to_file('/tmp/goat.gpkg')

In [ ]:
#livestock_rows_df[np.logical_and.reduce([
#    livestock_rows_df['animal'] == 'goat',
#    livestock_rows_df['source'] == 'GPW',
#    livestock_rows_df['total'] < 500
#])][['eml_msum','total']].plot.hist(bins=64, histtype='step', log=True, linewidth=1)

In [ ]:
pred_intervals_check = []

max_density = {
  'cattle': 4271,
    'sheep': 344,
    'goat': 279,
    'horse': 42,
    'buffalo': 290
}

for a in animals:
    animal_df = livestock_rows_df[np.logical_and.reduce([
        livestock_rows_df['animal'] == a,
        livestock_rows_df['source'] == 'GPW',
        livestock_rows_df['density'] < max_density[a]
    ])]
    animal_df.loc[:,'total_pixels'] = animal_df['total_pixels'] / animal_df['total_pixels'].sum()
    pred_intervals_check.append(
        {
            'Animal': a.title(),
            'Inside': ((animal_df['inside_pred_interval'] == 1).astype('int') * animal_df['total_pixels']).sum()
        }
    )

pd.DataFrame(pred_intervals_check).set_index('Animal').plot(kind='barh', legend=False, xlabel="Prediction Interval Coverage Probability")

In [ ]:
#%%
#https://rowannicholls.github.io/python/statistics/agreement/concordance_correlation_coefficient.html
#https://github.com/stylianos-kampakis/supervisedPCA-Python/blob/master/Untitled.py
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer
from statsmodels.stats.weightstats import DescrStatsW

def NormRootMeanSqrtErr(y_true, y_pred, type = "minmax"):
    """
    Normalized Root Mean Square Error.
    
    interpretation: smaller is better.

    Args:
        y_true ([np.array]): test samples
        y_pred ([np.array]): predicted samples
        type (str): type of normalization. default is "minmax"
        - sd (standard deviation) : it divides the value by the standard deviation of the data.
        - mean (mean) : it divides the value by the mean of the data.
        - minmax (min-max) : it divides the value by the range of the data.
        - iqr (interquartile range) : it divides the value by the interquartile range of the data.

    Returns:
        [float]: normalized root mean square error
    """
    if type=="sd":
        return np.sqrt(mean_squared_error(y_true, y_pred))/np.std(y_true)
    elif type=="mean":
        return np.sqrt(mean_squared_error(y_true, y_pred))/np.mean(y_true)
    elif type=="minmax":
        return np.sqrt(mean_squared_error(y_true, y_pred))/(np.max(y_true) - np.min(y_true))
    elif type=="iqr":
        return np.sqrt(mean_squared_error(y_true, y_pred))/(np.quantile(y_true, 0.75) - np.quantile(y_true, 0.25))
    elif type!="":
        raise ValueError("type must be either 'sd', 'mean', 'minmax', or 'iqr'")

def concordance_correlation_coefficient(y_true, y_pred):
    """Concordance correlation coefficient."""
    # Raw data
    dct = {
        'y_true': y_true,
        'y_pred': y_pred
    }
    wei = np.ones(len(y_true))
    df = pd.DataFrame(dct)
    # Remove NaNs
    df = df.dropna()
    # Pearson product-moment correlation coefficients
    y_true = df['y_true']
    y_pred = df['y_pred']
    #cor = np.corrcoef(y_true, y_pred)[0][1]
    cor = DescrStatsW(df.to_numpy(), weights=wei).corrcoef[0][1]
    # Means
    mean_true = np.average(y_true, weights=wei)
    mean_pred = np.average(y_pred, weights=wei)
    # Population variances
    #var_true = np.var(y_true)
    var_true = DescrStatsW(y_true, weights=wei).var
    #print(var_true, var_true1)
    #var_pred = np.var(y_pred)
    var_pred = DescrStatsW(y_pred, weights=wei).var
    # Population standard deviations
    sd_true = DescrStatsW(y_true, weights=wei).std
    sd_pred = DescrStatsW(y_pred, weights=wei).std
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2

    return numerator / denominator

ccc_scorer = make_scorer(concordance_correlation_coefficient, greater_is_better=True)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error, d2_tweedie_score

import math
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid", palette="deep")
sns.set_context("notebook")

max_density = {
    'cattle': 4271,
    'sheep': 344,
    'goat': 279,
    'horse': 42,
    'buffalo': 290
}

def compute_accuracy(df, predicted_col, animal, max_density):
    mask = np.logical_and.reduce([
        ~np.isnan(df[predicted_col]), 
        df[predicted_col] > 0,
        df['animal'] == animal,
        df['source'] == 'GPW',
        df['density'] < max_density[animal],
    ])
    
    result = {}
    result[f'R2'] = r2_score(df[mask][expected_col], df[mask][predicted_col])
    result[f'NRMSE'] = NormRootMeanSqrtErr(df[mask][expected_col], df[mask][predicted_col], 'sd')
    result[f'MSLE'] = mean_squared_log_error(df[mask][expected_col], df[mask][predicted_col])
    result[f'MAPE'] = mean_absolute_percentage_error(df[mask][expected_col], df[mask][predicted_col])
    result[f'D2'] = d2_tweedie_score(df[mask][expected_col], df[mask][predicted_col], power=1.5)
    result[f'CCC'] = concordance_correlation_coefficient(df[mask][expected_col], df[mask][predicted_col])
    result['Animal'] = animal.title()
    result['Model'] = predicted_col
    
    return result

def plot_accuracy(acc_result, metrics, mscales):
    n_metrics = len(metrics)
    fig, axs = plt.subplots(ncols=n_metrics, figsize=(n_metrics*3,2))
    plt.tight_layout(pad=-0.5)

    for i, metric, mscale in zip(range(0,len(metrics)), metrics, mscales):
        sns.barplot(x=metric, y='Animal', hue='Model', data=acc_result, ax=axs[i], alpha=0.9)
        scale = 1 / mscale
        mix_tick, max_tick = math.floor(acc_result[metric].min() * scale) / scale, math.ceil(acc_result[metric].max() * scale) / scale
        if i > 0:
            axs[i].set(yticklabels=[], ylabel="")
            axs[i].legend_.remove()
        else:
            sns.move_legend(axs[i], "center left", bbox_to_anchor=(-0.03, 1.10), ncol=3, title='')
        #axs[i].set_xticks(np.arange(0, max_tick+mscale, mscale))

expected_col = 'total'
cols = ['eml_msum', 'rf_msum', 'lgb_msum']
#animals = ['cattle', 'buffalo', 'horse']
animals = ['goat', 'sheep']
col_remap = {
    'eml_msum': 'Ensemble',
    'rf_msum': 'Random forest',
    'lgb_msum': 'Gradient-boosted trees',
}

metrics = ['R2', 'CCC', 'NRMSE']
mscales = [0.1, 0.1, 0.25]

acc_result = []
for animal in animals:
    for c in cols:
        acc_result.append(compute_accuracy(livestock_rows_df, c, animal, max_density))
        
acc_result = pd.DataFrame(acc_result)
acc_result['Model'] = acc_result['Model'].map(col_remap)

plot_accuracy(acc_result, metrics, mscales)